## Bem vindo a Pokedex de batalha.

Aqui, com o intuito de montar um time competitivo de batalha, para qualquer geração da franquia, apresento uma ferramenta de busca tanto de pokémons, quanto de movimentos e TMs.

### Documentação do Código
Descrição
Este script coleta dados de três diferentes endpoints da API do Pokémon (PokéAPI), armazena esses dados em um banco de dados, realiza algumas manipulações básicas nos dados e o converte para o formato CSV. O script é dividido em três partes principais:

### Coleta de dados de Pokémons.
### Coleta de dados de TMs. 
### Coleta de dados de Movimentos.

Dependências: Este script depende das seguintes bibliotecas: pandas requests sqlite3 datetime

### Coleta de Dados

Coleta de Dados dos Pokémons Coleta dados dos Pokémons da PokéAPI e salva no banco de dados. Dados de cada espécie de pokémon, são retirados da API "https://pokeapi.co/api/v2/pokemon", e categorizados em: Name - Nome da espécie Height - Altura do pokémon (importante Weight - Peso do pokémon (importante para moves como Low Kick e Grass Knot) Type - O tipo de cada pokémon, podendo ter até dois Base Experience - Parâmetro que influencia na taxa de crescimento do pokémon

Coleta de Dados de TMs Coleta dados de TMs da PokéAPI e salva no banco de dados. Dados de cada TM de versões diferentes do jogo, retirado da API "https://pokeapi.co/api/v2/machine", e categorizados em: TM: Número da TM Generation: Geração da franquia ao qual esta TM está listada Move: Nome do movimento

Coleta de Dados de Movimentos Coleta dados de Movimentos da PokéAPI e salva no banco de dados. Dados de cada movimentos de pokémons, retirados da API "https://pokeapi.co/api/v2/move" e categorizados em: Nome: Nome Type: Tipo Power: Poder base Accuracy: Precisão Power Points: PP (número de usos) Effect: Pequena descrição do movimento e seus efeitos secundários.

Consulta das Tabelas no Banco de Dados Consulta e exibe os nomes das tabelas presentes no banco de dados.

## Conclusão
Este script fornece uma maneira automatizada de coletar, processar e armazenar dados da PokéAPI em um banco de dados CSV, tornando possível seu consumo no SQL Server para uma análise aprofundada de dados.

Importando Bibliotecas

In [26]:
import pandas as pd
import requests
import sqlite3

Função para salvar o banco de dados

In [2]:
def save_to_db(df, table_name, conn):
    df.to_sql(table_name, conn, if_exists='replace', index=False)
    print(f"Tabela {table_name} salva no banco de dados")

# Pokemons

Consumo de APIs e criação do Data Frame

In [3]:
url = "https://pokeapi.co/api/v2/pokemon"
response = requests.get(url)

In [4]:
#verificação da resposta da funcionalidade da API
if response.status_code == 200:
    data_json = response.json()
    print("Funcionou")

    pokemon_list = []
    for pokemon in data_json['results']:
        pokemon_url = pokemon['url']
        pokemon_response = requests.get(pokemon_url)
        if pokemon_response.status_code == 200:
            pokemon_data = pokemon_response.json()            
        
            types = ", ".join([type_info['type']['name'] for type_info in pokemon_data['types']])
            
            pokemon_info = {
                'name': pokemon_data['name'],
                'height': pokemon_data['height'],
                'weight': pokemon_data['weight'],
                'types': types,
                'base_experience': pokemon_data['base_experience']
            }
            pokemon_list.append(pokemon_info)
        else:
            print(f"Falha ao acessar dados de {pokemon['name']}")

    df_pokemons = pd.DataFrame(pokemon_list)
else:
    print("Não foi")

Funcionou


Limpeza de dados

In [5]:
df_pokemons.duplicated()
df_pokemons.drop_duplicates()

,name,height,weight,types,base_experience
0,bulbasaur,7,69,"grass, poison",64
1,ivysaur,10,130,"grass, poison",142
2,venusaur,20,1000,"grass, poison",236
3,charmander,6,85,fire,62
4,charmeleon,11,190,fire,142
5,charizard,17,905,"fire, flying",240
6,squirtle,5,90,water,63
7,wartortle,10,225,water,142
8,blastoise,16,855,water,239
9,caterpie,3,29,bug,39


In [6]:
df_pokemons.groupby('height')
df_pokemons.sort_values('height', ascending=True)

,name,height,weight,types,base_experience
12,weedle,3,32,"bug, poison",39
15,pidgey,3,18,"normal, flying",50
9,caterpie,3,29,bug,39
18,rattata,3,35,normal,51
6,squirtle,5,90,water,63
13,kakuna,6,100,"bug, poison",72
3,charmander,6,85,fire,62
0,bulbasaur,7,69,"grass, poison",64
19,raticate,7,185,normal,145
10,metapod,7,99,bug,72


Criação de Banco de Dados e conexão com SQL

In [7]:
conn = sqlite3.connect('pokedex.db')
save_to_db(df_pokemons, 'Pokemon', conn)
conn.close()

Tabela Pokemon salva no banco de dados


In [9]:
conn = sqlite3.connect('pokedex.db')
query = "SELECT * FROM Pokemon"
df = pd.read_sql(query, conn)
print(df)

conn.close()

          name  height  weight           types  base_experience
0    bulbasaur       7      69   grass, poison               64
1      ivysaur      10     130   grass, poison              142
2     venusaur      20    1000   grass, poison              236
3   charmander       6      85            fire               62
4   charmeleon      11     190            fire              142
5    charizard      17     905    fire, flying              240
6     squirtle       5      90           water               63
7    wartortle      10     225           water              142
8    blastoise      16     855           water              239
9     caterpie       3      29             bug               39
10     metapod       7      99             bug               72
11  butterfree      11     320     bug, flying              178
12      weedle       3      32     bug, poison               39
13      kakuna       6     100     bug, poison               72
14    beedrill      10     295     bug, 

# TMs

Consumo de APIs e criação do Data Frame

In [10]:
url = "https://pokeapi.co/api/v2/machine"
response = requests.get(url)

In [11]:
if response.status_code == 200:
    data_json = response.json()
    print("Funcionou")

    machine_list = []
    for machine in data_json['results']:
        machine_url = machine['url']
        machine_response = requests.get(machine_url)
        machine_data = machine_response.json()
        machine_info = {
             'id': machine_data['id'],
             'TM': machine_data['item']['name'],
             'generation': machine_data['version_group']['name'],
             'move': machine_data['move']['name'],

            }
        machine_list.append(machine_info)
    df_machine_list = pd.DataFrame(machine_list)
    
else:
    print("Não foi")

Funcionou


Limpeza de dados

In [12]:
df_machine_list.duplicated()
df_machine_list.drop_duplicates()

,id,TM,generation,move
0,1,tm00,sword-shield,mega-punch
1,2,tm01,red-blue,mega-punch
2,3,tm01,yellow,mega-punch
3,4,tm01,gold-silver,dynamic-punch
4,5,tm01,crystal,dynamic-punch
5,6,tm01,ruby-sapphire,focus-punch
6,7,tm01,emerald,focus-punch
7,8,tm01,firered-leafgreen,focus-punch
8,9,tm01,diamond-pearl,focus-punch
9,10,tm01,platinum,focus-punch


Criação de Banco de Dados e conexão com SQL

In [22]:
conn = sqlite3.connect('pokedex.db')
save_to_db(df_machine_list, 'TMs', conn)
conn.close()

Tabela TMs salva no banco de dados


In [23]:
conn = sqlite3.connect('pokedex.db')
query = "SELECT * FROM TMs"
df = pd.read_sql(query, conn)
print(df)

conn.close()

    id    TM                     generation           move
0    1  tm00                   sword-shield     mega-punch
1    2  tm01                       red-blue     mega-punch
2    3  tm01                         yellow     mega-punch
3    4  tm01                    gold-silver  dynamic-punch
4    5  tm01                        crystal  dynamic-punch
5    6  tm01                  ruby-sapphire    focus-punch
6    7  tm01                        emerald    focus-punch
7    8  tm01              firered-leafgreen    focus-punch
8    9  tm01                  diamond-pearl    focus-punch
9   10  tm01                       platinum    focus-punch
10  11  tm01           heartgold-soulsilver    focus-punch
11  12  tm01                    black-white     hone-claws
12  13  tm01                      colosseum    focus-punch
13  14  tm01                             xd    focus-punch
14  15  tm01                black-2-white-2     hone-claws
15  16  tm01                            x-y     hone-cla

# Moves

Consumo de APIs e criação do Data Frame

In [15]:
url = "https://pokeapi.co/api/v2/move"
response = requests.get(url)

In [16]:
if response.status_code == 200:
    data_json = response.json()
    print("funcionou")

    move_list = []
    for move in data_json ['results']:
        move_url = move['url']
        move_response = requests.get(move_url)
        move_data = move_response.json()
        move_info = {
            'name': move_data ['name'],
            'type': move_data ['type']['name'],
            'power': move_data['power'],
            'accuracy': move_data['accuracy'],
            'power points': move_data['pp'],
            'effect': move_data ['effect_entries'][0]['short_effect'],
        }
        move_list.append(move_info)
    df_moves = pd.DataFrame(move_list)

else:
    print("nao funcionou")

funcionou


Limpeza de dados

In [17]:
df_moves.duplicated()
df_moves.drop_duplicates

<bound method DataFrame.drop_duplicates of              name      type  power  accuracy  power points  \
0           pound    normal   40.0     100.0            35   
1     karate-chop  fighting   50.0     100.0            25   
2     double-slap    normal   15.0      85.0            10   
3     comet-punch    normal   18.0      85.0            15   
4      mega-punch    normal   80.0      85.0            20   
5         pay-day    normal   40.0     100.0            20   
6      fire-punch      fire   75.0     100.0            15   
7       ice-punch       ice   75.0     100.0            15   
8   thunder-punch  electric   75.0     100.0            15   
9         scratch    normal   40.0     100.0            35   
10      vice-grip    normal   55.0     100.0            30   
11     guillotine    normal    NaN      30.0             5   
12     razor-wind    normal   80.0     100.0            10   
13   swords-dance    normal    NaN       NaN            20   
14            cut    normal

In [19]:
conn = sqlite3.connect('pokedex.db')
save_to_db(df_moves, 'Moves', conn)
conn.close()

Tabela Moves salva no banco de dados


Criação de Banco de Dados e conexão com SQL

In [20]:
conn = sqlite3.connect('pokedex.db')
query = "SELECT * FROM Moves"
df = pd.read_sql(query, conn)
print(df)

conn.close()

             name      type  power  accuracy  power points  \
0           pound    normal   40.0     100.0            35   
1     karate-chop  fighting   50.0     100.0            25   
2     double-slap    normal   15.0      85.0            10   
3     comet-punch    normal   18.0      85.0            15   
4      mega-punch    normal   80.0      85.0            20   
5         pay-day    normal   40.0     100.0            20   
6      fire-punch      fire   75.0     100.0            15   
7       ice-punch       ice   75.0     100.0            15   
8   thunder-punch  electric   75.0     100.0            15   
9         scratch    normal   40.0     100.0            35   
10      vice-grip    normal   55.0     100.0            30   
11     guillotine    normal    NaN      30.0             5   
12     razor-wind    normal   80.0     100.0            10   
13   swords-dance    normal    NaN       NaN            20   
14            cut    normal   50.0      95.0            30   
15      

Verificação dos bancos de dados criados para consumo no SQL

In [24]:
conn = sqlite3.connect('pokedex.db')
query = "SELECT name FROM sqlite_master WHERE type='table'"
schema = pd.read_sql_query(query, conn)
print(schema)

conn.close()

      name
0  Pokemon
1    Moves
2      TMs
